In [ ]:
print('Exporting data...')

In [1]:
import boto3
from botocore import UNSIGNED
from botocore.client import Config
from botocore.exceptions import NoCredentialsError, PartialCredentialsError, EndpointConnectionError
import psycopg2
from psycopg2 import sql
import csv
from dotenv import load_dotenv
import os

print('All dependencies have been imported.')

In [15]:
# Creating an S3 client
s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))

# Bucket name to upload the file
bucket_name = "d2b-internal-assessment-bucket"


print('Connected to s3 Bucket.')

In [3]:
# Load environmental variables from the .env file
load_dotenv("/workspaces/Data2Bots-Assessment/.env")

# Get the environmental variables
PG_USERNAME = os.getenv("PG_USERNAME")
PG_PASS = os.getenv("PG_PASS")
PG_HOST = os.getenv("PG_HOST")
PG_DB = os.getenv("PG_DB")

In [4]:
# PostgreSQL database connection parameters
db_params = {
    'host': PG_HOST,
    'port': 5432,
    'database': PG_DB,
    'user': PG_USERNAME,
    'password': PG_PASS
}


connection = psycopg2.connect(**db_params)
cursor = connection.cursor()

print('Database connection successfully.')

In [ ]:

# Query to get the list of tables in the schema
tables_query = f"SELECT table_name FROM information_schema.tables WHERE table_schema = 'joshodey2178_analytics';"
cursor.execute(tables_query)
tables = cursor.fetchall()

# Loop to get table name, and create empty .csv file
for table in tables:
    table_name = table[0]
    csv_file_path = f"/workspaces/Data2Bots-Assessment/table_exports/{table_name}.csv"
    
    # Fetch data from the PostgreSQL table
    data_query = f"SELECT * FROM joshodey2178_analytics.{table_name};"
    cursor.execute(data_query)
    result = cursor.fetchall()
    
    # Write data to the empty CSV file created
    with open(csv_file_path, 'w', newline='') as csv_file:
        csv_writer = csv.writer(csv_file)
        # Write the column names
        csv_writer.writerow([desc[0] for desc in cursor.description])
        # Write the rows
        csv_writer.writerows(result)

    print(f"Data from table {table_name} exported to path")

In [ ]:
# Specify the local directory containing CSV files
local_directory = "/workspaces/Data2Bots-Assessment/table_exports/"

try:
    # Loop through each file in the local directory
    for filename in os.listdir(local_directory):
        if filename.endswith(".csv"):
            local_file_path = os.path.join(local_directory, filename)
            key = f"analytics_export/joshodey2178/{filename}"  # S3 key based on the filename

            # Upload the file to S3
            s3.upload_file(local_file_path, bucket_name, key)
            print(f"File {filename} uploaded to s3://{bucket_name}/{key}")

except FileNotFoundError:
    print(f"One or more files in {local_directory} were not found. Please check the file paths.")

    
